## PyTorch Embedding for High Cardinality features
torch.nn.Embedding is a layer in PyTorch that maps each input index (usually representing a word or a token) to a fixed-size vector. This vector is learned during the training process and captures the semantic meaning of the input.

Usage:

 - Natural Language Processing (NLP): torch.nn.Embedding is commonly used in NLP tasks where words or tokens need to be represented as dense vectors. Each word or token is mapped to a vector of continuous values, and similar words often end up with similar vector representations.
 - Categorical Features with Many Categories: When dealing with categorical features that have a large number of categories, torch.nn.Embedding can efficiently map these categories to lower-dimensional dense vectors.


Advantages:

- Dimensionality Reduction: Instead of using a high-dimensional one-hot vector, torch.nn.Embedding reduces the dimensionality to a fixed-size vector, which is more efficient in terms of memory and computation.
- Learned Representations: The embedding vectors are learned during training and can capture relationships between different categories (e.g., semantic similarity between words).

The embeddings are better for high cardinality datasets like zipcodes than one hot encoded vectors.

In [7]:
import torch
import torch.nn as nn
torch.manual_seed(42)

In [35]:
embedding = nn.Embedding(num_embeddings=11, embedding_dim=5)
input = torch.LongTensor([8, 1, 0, 5, 10])
output = embedding(input)
print(output)

tensor([[ 0.1011, -1.3095, -0.4104,  0.4681, -0.2346],
        [-0.7888,  1.3062, -0.9276, -0.2627,  0.9315],
        [-0.4477, -0.7288, -0.1607, -0.3206, -0.6308],
        [ 0.6950,  1.8352, -1.9180, -1.3924,  0.5405],
        [-1.7025,  1.6696, -0.1214,  0.8139, -0.3816]],
       grad_fn=<EmbeddingBackward0>)


#### How to retrieve the elements of input tensor from the embedding?
The embedding object contains a column vector of num_embeddings size contaning all the representations done for the input. We can reach this vector with embedding.weight.data. The position of the corresponding output vector in this column vector is the corresponding input vector member. In example to retrieve the first element (index=0) of the input vector (8):

In [36]:
embb_weights = embedding.weight.data
print(embb_weights)

tensor([[-4.4770e-01, -7.2882e-01, -1.6066e-01, -3.2064e-01, -6.3077e-01],
        [-7.8877e-01,  1.3062e+00, -9.2758e-01, -2.6274e-01,  9.3150e-01],
        [-4.5935e-01, -9.4195e-01, -7.0892e-01,  2.1861e+00, -6.4932e-01],
        [ 4.5214e-01,  8.5207e-01, -1.6947e+00,  1.1806e+00, -2.8929e+00],
        [-3.8758e-01, -7.1240e-01, -1.6171e+00, -3.5899e-01,  5.1367e-02],
        [ 6.9502e-01,  1.8352e+00, -1.9180e+00, -1.3924e+00,  5.4047e-01],
        [ 4.3507e-01, -2.2717e+00, -1.3386e-01, -5.8557e-02,  1.2574e-01],
        [-5.5258e-01,  7.4480e-02, -1.4929e-01, -5.5225e-01,  7.0114e-01],
        [ 1.0107e-01, -1.3095e+00, -4.1036e-01,  4.6809e-01, -2.3463e-01],
        [ 2.7824e-01, -1.2937e-04,  1.5818e+00, -5.5852e-01,  6.6369e-01],
        [-1.7025e+00,  1.6696e+00, -1.2137e-01,  8.1387e-01, -3.8159e-01]])


In [37]:
# distance is a new vector containing the Euclidean norm, or distance, between each member of the weights column vector and the corresponding representation of 8 in the output 
distance = torch.norm(embb_weights - output[0], dim=1)
print(distance)

tensor([1.2164, 3.1297, 1.9136, 3.7442, 1.6785, 4.0715, 1.2333, 2.0803, 0.0000,
        2.7523, 3.5146], grad_fn=<LinalgVectorNormBackward0>)


In [38]:
# the index where is the minor or zero value of the distance is the value of the first member of the input vector
closest_index = torch.argmin(distance).item()
print(closest_index)

8


#### Function to recover the input
Important Considerations:

- Approximation: This method approximates the reverse mapping, as the learned embedding vectors are not necessarily unique or perfectly reversible.
- Distance Metric: The choice of distance metric (e.g., Euclidean, cosine) may affect the results. Different metrics can yield different "closest" indices.
- Context Sensitivity: In real-world applications, embedding vectors may also depend on the surrounding context (as in word embeddings), making a perfect inverse mapping infeasible.

In [40]:
# Function to find the closest index for each embedding vector in the output
def find_closest_indices(embedding_layer, vectors):
    embedding_weights = embedding_layer.weight.data
    closest_indices = []
    for vector in vectors:
        distances = torch.norm(embedding_weights - vector, dim=1)
        closest_index = torch.argmin(distances).item()
        closest_indices.append(closest_index)
    return closest_indices

recovered_input_as_list = find_closest_indices(embedding, output)
print(recovered_input_as_list)

[8, 1, 0, 5, 10]
